<a href="https://colab.research.google.com/github/Aman05gupta/Comparative-Analysis-of-Adversarial-Attack-Techniques-and-Defense-Mechanisms/blob/main/RBL_Project_Data__Poisoning_Attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install required libraries

In [6]:
pip install --upgrade adversarial-robustness-toolbox


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.4 MB/s eta 0:00:00


In [2]:
!pip install tensorflow keras foolbox art==1.10.0 matplotlib

ERROR: Could not find a version that satisfies the requirement art==1.10.0 (from versions: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3)
ERROR: No matching distribution found for art==1.10.0


In [48]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import KerasClassifier
from art.defences.preprocessor import FeatureSqueezing


# 2. Load and preprocess the dataset

In [12]:
mnist.load_data()

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Normalize the data to [0, 1] range

In [13]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape data to add a channel dimension

In [15]:
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

# 3. Build a simple CNN model

In [16]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model

In [18]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 53s 61ms/step - accuracy: 0.8727 - loss: 0.4285 - val_accuracy: 0.9835 - val_loss: 0.0572
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 88s 68ms/step - accuracy: 0.9808 - loss: 0.0639 - val_accuracy: 0.9872 - val_loss: 0.0450
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 76s 61ms/step - accuracy: 0.9861 - loss: 0.0441 - val_accuracy: 0.9883 - val_loss: 0.0370
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 83s 63ms/step - accuracy: 0.9914 - loss: 0.0285 - val_accuracy: 0.9898 - val_loss: 0.0364
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 81s 62ms/step - accuracy: 0.9928 - loss: 0.0219 - val_accuracy: 0.9863 - val_loss: 0.0515


# Evaluate the model on clean test data

In [20]:
clean_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"Clean test accuracy: {clean_accuracy:.2f}")

Clean test accuracy: 0.99


# 4. Wrap the model with ART's KerasClassifier

In [43]:
!pip install --upgrade adversarial-robustness-toolbox

In [42]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [45]:
from art.estimators.classification import TensorFlowV2Classifier

# ... your code ...

# Assuming 'model' is your Keras model
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=10, # Replace with the actual number of classes
    input_shape=(28, 28, 1), # Replace with the actual input shape
    loss_object=tf.keras.losses.CategoricalCrossentropy(), # Replace with your actual loss function
    clip_values=(0, 1)
)

# 5. Create adversarial examples using FGSM

# 6. Create adversarial examples using PGD

# 8. Visualization
# Plot a few original and adversarial images for comparison